# Section 6: Forecasting and Practical Applications

#### PyData London 2025 - Bayesian Time Series Analysis with PyMC

---

## The Power of Probabilistic Forecasting

Traditional forecasting methods provide point estimates with confidence intervals based on strong assumptions. **Bayesian forecasting** offers several advantages:

### Key Benefits

1. **Full Uncertainty Quantification**: Complete probability distributions over future values
2. **Coherent Uncertainty Propagation**: Parameter uncertainty flows naturally into forecasts
3. **Flexible Prediction Intervals**: Any quantile or probability can be computed
4. **Decision-Theoretic Framework**: Optimal decisions under uncertainty

### Forecast Types

- **Point forecasts**: Expected values (posterior means)
- **Interval forecasts**: Credible intervals (e.g., 90% HDI)
- **Probabilistic forecasts**: Full predictive distributions
- **Scenario analysis**: Probability of specific events

In [ ]:
# Import necessary libraries for Section 6
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import pymc as pm
import arviz as az
import warnings

# Configure plotting and suppress warnings
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 100
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
RANDOM_SEED = 42

print("🔧 Section 6 libraries loaded successfully!")
print("Ready to generate probabilistic forecasts!")

## Data Preparation and Train-Test Split

For proper forecast evaluation, we need to split our data into training and testing sets.

In [ ]:
# Load and prepare data with train-test split
births_data = pl.read_csv('../data/births.csv', null_values=['null', 'NA', '', 'NULL'])
births_data = births_data.filter(pl.col('day').is_not_null())

monthly_births = (births_data
    .group_by(['year', 'month'])
    .agg(pl.col('births').sum())
    .sort(['year', 'month'])
)

births_subset = (monthly_births
    .filter((pl.col('year') >= 1970) & (pl.col('year') <= 1990))
    .with_row_index('index')
)

# Standardize the data
original_data = births_subset['births'].to_numpy()
births_standardized = (original_data - original_data.mean()) / original_data.std()

# Train-test split (use last 24 months for testing)
n_forecast = 24
n_train = len(births_standardized) - n_forecast

y_train = births_standardized[:n_train]
y_test = births_standardized[n_train:]

print(f"📊 Data split:")
print(f"   Training: {len(y_train)} observations")
print(f"   Testing: {len(y_test)} observations")
print(f"   Forecast horizon: {n_forecast} months")

## Building a Forecasting Model

Let's build a seasonal model that can generate forecasts with uncertainty quantification.

In [ ]:
# Build a seasonal model for forecasting
time_idx_train = np.arange(len(y_train))
time_normalized_train = (time_idx_train - time_idx_train.mean()) / time_idx_train.std()

# Create seasonal components for training data
period = 12  # Monthly data with annual seasonality
seasonal_freq = 2 * np.pi / period
sin_seasonal_train = np.sin(seasonal_freq * time_idx_train)
cos_seasonal_train = np.cos(seasonal_freq * time_idx_train)

with pm.Model() as forecast_model:
    # Model parameters
    mu_overall = pm.Normal('mu_overall', mu=0, sigma=1)
    beta_trend = pm.Normal('beta_trend', mu=0, sigma=1)
    beta_sin = pm.Normal('beta_sin', mu=0, sigma=1)
    beta_cos = pm.Normal('beta_cos', mu=0, sigma=1)
    
    # Expected value for training data
    mu_train = pm.Deterministic('mu_train', 
                               mu_overall + 
                               beta_trend * time_normalized_train + 
                               beta_sin * sin_seasonal_train + 
                               beta_cos * cos_seasonal_train)
    
    # Observation noise
    sigma_obs = pm.HalfNormal('sigma_obs', sigma=1)
    
    # Likelihood (training data only)
    y_obs = pm.Normal('y_obs', mu=mu_train, sigma=sigma_obs, observed=y_train)

# Fit the model
with forecast_model:
    trace_forecast = pm.sample(1000, tune=1000, random_seed=RANDOM_SEED, chains=2)

print("✅ Forecasting model fitted successfully")
print(az.summary(trace_forecast, var_names=['mu_overall', 'beta_trend', 'beta_sin', 'beta_cos', 'sigma_obs']))

## Generating Probabilistic Forecasts

Now let's generate forecasts for the test period with full uncertainty quantification.

In [ ]:
# Generate forecasts for the test period
time_idx_forecast = np.arange(len(y_train), len(y_train) + n_forecast)
time_normalized_forecast = (time_idx_forecast - time_idx_train.mean()) / time_idx_train.std()

# Create seasonal components for forecast period
sin_seasonal_forecast = np.sin(seasonal_freq * time_idx_forecast)
cos_seasonal_forecast = np.cos(seasonal_freq * time_idx_forecast)

# Generate forecasts using posterior samples
with forecast_model:
    # Compute expected values for forecast period
    mu_forecast = pm.Deterministic('mu_forecast',
                                  trace_forecast.posterior['mu_overall'] +
                                  trace_forecast.posterior['beta_trend'] * time_normalized_forecast +
                                  trace_forecast.posterior['beta_sin'] * sin_seasonal_forecast +
                                  trace_forecast.posterior['beta_cos'] * cos_seasonal_forecast)
    
    # Generate predictive samples
    y_forecast = pm.Normal('y_forecast', 
                          mu=mu_forecast, 
                          sigma=trace_forecast.posterior['sigma_obs'])
    
    # Sample from posterior predictive distribution
    forecast_samples = pm.sample_posterior_predictive(trace_forecast, 
                                                     var_names=['y_forecast'],
                                                     random_seed=RANDOM_SEED)

print("🔮 Generated probabilistic forecasts for test period")

In [ ]:
# Extract forecast statistics
forecast_data = forecast_samples.posterior_predictive['y_forecast']

# Compute forecast statistics
forecast_mean = forecast_data.mean(dim=['chain', 'draw'])
forecast_median = forecast_data.median(dim=['chain', 'draw'])
forecast_hdi_90 = az.hdi(forecast_data, hdi_prob=0.9)
forecast_hdi_50 = az.hdi(forecast_data, hdi_prob=0.5)

print("📊 Forecast Statistics Computed:")
print(f"   Mean forecast: {forecast_mean.values[:5]} ... (first 5 values)")
print(f"   90% HDI width: {(forecast_hdi_90.sel(hdi='higher') - forecast_hdi_90.sel(hdi='lower')).values[:5]} ... (first 5 values)")

## Forecast Visualization and Evaluation

Let's visualize our forecasts and evaluate their performance.

In [ ]:
# Create comprehensive forecast visualization
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))

# Full time series with forecasts
time_all = np.arange(len(births_standardized))
time_train = time_all[:n_train]
time_test = time_all[n_train:]

# Plot training data
ax1.plot(time_train, y_train, 'o-', color='blue', alpha=0.7, label='Training Data')

# Plot test data (actual)
ax1.plot(time_test, y_test, 'o-', color='red', alpha=0.7, label='Actual (Test)')

# Plot forecasts
ax1.plot(time_test, forecast_mean, 's-', color='green', linewidth=2, label='Forecast Mean')

# Plot uncertainty bands
ax1.fill_between(time_test, 
                forecast_hdi_90.sel(hdi='lower'), 
                forecast_hdi_90.sel(hdi='higher'), 
                alpha=0.3, color='green', label='90% HDI')

ax1.fill_between(time_test, 
                forecast_hdi_50.sel(hdi='lower'), 
                forecast_hdi_50.sel(hdi='higher'), 
                alpha=0.5, color='green', label='50% HDI')

ax1.axvline(n_train, color='black', linestyle='--', alpha=0.7, label='Train/Test Split')
ax1.set_title('Bayesian Time Series Forecasting with Uncertainty')
ax1.set_xlabel('Time')
ax1.set_ylabel('Standardized Births')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Forecast errors
forecast_errors = y_test - forecast_mean.values
ax2.plot(time_test, forecast_errors, 'o-', color='purple', alpha=0.7)
ax2.axhline(0, color='black', linestyle='-', alpha=0.5)
ax2.set_title('Forecast Errors')
ax2.set_xlabel('Time')
ax2.set_ylabel('Error (Actual - Forecast)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Compute forecast evaluation metrics
mae = np.mean(np.abs(forecast_errors))
rmse = np.sqrt(np.mean(forecast_errors**2))
mape = np.mean(np.abs(forecast_errors / y_test)) * 100

print(f"\n📈 **Forecast Evaluation Metrics**:")
print(f"   Mean Absolute Error (MAE): {mae:.4f}")
print(f"   Root Mean Square Error (RMSE): {rmse:.4f}")
print(f"   Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

# Coverage analysis
in_90_hdi = ((y_test >= forecast_hdi_90.sel(hdi='lower').values) & 
             (y_test <= forecast_hdi_90.sel(hdi='higher').values))
coverage_90 = np.mean(in_90_hdi) * 100

in_50_hdi = ((y_test >= forecast_hdi_50.sel(hdi='lower').values) & 
             (y_test <= forecast_hdi_50.sel(hdi='higher').values))
coverage_50 = np.mean(in_50_hdi) * 100

print(f"\n🎯 **Coverage Analysis**:")
print(f"   90% HDI Coverage: {coverage_90:.1f}% (target: 90%)")
print(f"   50% HDI Coverage: {coverage_50:.1f}% (target: 50%)")

## Practical Applications and Decision Making

Bayesian forecasts enable sophisticated decision-making under uncertainty.

In [ ]:
# Practical applications: Probabilistic decision making
print("🎯 **Practical Applications: Decision Making Under Uncertainty**")
print("="*70)

# Example 1: Probability of exceeding thresholds
threshold_high = 1.0  # High birth count threshold
threshold_low = -1.0  # Low birth count threshold

prob_exceed_high = (forecast_data > threshold_high).mean(dim=['chain', 'draw'])
prob_below_low = (forecast_data < threshold_low).mean(dim=['chain', 'draw'])

print(f"\n**Threshold Analysis**:")
print(f"   Probability of exceeding {threshold_high}: {prob_exceed_high.values[:6]} (first 6 months)")
print(f"   Probability of falling below {threshold_low}: {prob_below_low.values[:6]} (first 6 months)")

# Example 2: Risk assessment
risk_months = np.where(prob_exceed_high > 0.7)[0]
if len(risk_months) > 0:
    print(f"\n**Risk Assessment**:")
    print(f"   High-risk months (>70% chance of exceeding threshold): {risk_months + 1}")
else:
    print(f"\n**Risk Assessment**: No high-risk months identified")

# Example 3: Quantile forecasts for planning
quantiles = [0.1, 0.25, 0.5, 0.75, 0.9]
forecast_quantiles = forecast_data.quantile(quantiles, dim=['chain', 'draw'])

print(f"\n**Quantile Forecasts for Planning** (first 3 months):")
for i, q in enumerate(quantiles):
    values = forecast_quantiles.isel(quantile=i).values[:3]
    print(f"   {q*100:4.0f}th percentile: {values}")

print(f"\n💡 **Business Applications**:")
print(f"   • **Capacity Planning**: Use 90th percentile for resource allocation")
print(f"   • **Risk Management**: Monitor probabilities of extreme events")
print(f"   • **Inventory Management**: Optimize stock levels using full distribution")
print(f"   • **Financial Planning**: Budget based on uncertainty ranges")

## Best Practices for Bayesian Forecasting

### Model Selection and Validation
1. **Use proper scoring rules** (e.g., CRPS) for probabilistic forecast evaluation
2. **Implement rolling window validation** for robust performance assessment
3. **Check forecast calibration** - do 90% intervals contain 90% of observations?
4. **Monitor model performance** over time and retrain when necessary

### Communication and Interpretation
1. **Visualize uncertainty clearly** using fan charts or probability bands
2. **Provide multiple scenarios** (optimistic, realistic, pessimistic)
3. **Explain probabilistic language** to non-technical stakeholders
4. **Focus on decision-relevant metrics** rather than just point forecasts

### Technical Considerations
1. **Account for model uncertainty** through model averaging
2. **Handle structural breaks** with time-varying parameters
3. **Incorporate external information** through informative priors
4. **Scale computations** for high-frequency or multivariate data

## Summary and Next Steps

### What We've Accomplished

In this tutorial, we've covered:

1. **Section 1**: Motivation and time series fundamentals
2. **Section 2**: Bayesian inference and PyMC workflow
3. **Section 3**: Basic time series models (random walks, AR models)
4. **Section 4**: Advanced models (state-space, stochastic volatility, GPs)
5. **Section 5**: Model diagnostics and comparison
6. **Section 6**: Probabilistic forecasting and applications

### Key Takeaways

- **Bayesian methods provide natural uncertainty quantification** for time series
- **PyMC offers powerful tools** for building sophisticated time series models
- **Proper diagnostics are essential** for reliable inference
- **Probabilistic forecasts enable better decision-making** under uncertainty

### Further Learning

- **Advanced Topics**: Vector autoregressions, dynamic factor models, regime-switching models
- **Computational Methods**: Variational inference, particle filters, sequential Monte Carlo
- **Applications**: Finance, epidemiology, climate science, marketing mix modeling
- **Software**: Stan, TensorFlow Probability, Pyro for alternative implementations

---

**Thank you for joining this tutorial on Bayesian Time Series Analysis with PyMC!**

**Questions? Let's discuss!** 🚀